In [3]:
import geomloss
import torch

In [4]:
use_cuda = torch.cuda.is_available()
#  N.B.: We use float64 numbers to get nice limits when blur -> +infinity
dtype = torch.cuda.DoubleTensor if use_cuda else torch.DoubleTensor

In [11]:
X1 = torch.rand((9,9)).type(dtype=dtype)
X2 = torch.rand((9,9)).type(dtype)

In [12]:
print(X1.device,X2.device)


cuda:0 cuda:0


In [13]:
# sink = []
loss = geomloss.SamplesLoss(loss="sinkhorn",p=2,blur=0.01,scaling=0.99)
print(loss(X1,X2).item())


0.46474621072868116


In [14]:
N = X1.shape[0]
torch.ones(N).type_as(X1) / N

tensor([0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111],
       device='cuda:0', dtype=torch.float64)

In [19]:
loss(X1,X2)

tensor(0.4647, device='cuda:0', dtype=torch.float64)

In [9]:
dtype

torch.cuda.DoubleTensor

In [10]:
X1.device

device(type='cpu')

In [17]:
Y1 = torch.rand((9,9))
Y2 = torch.rand((9,9))

In [20]:
loss(Y1,Y2)

tensor(0.4475)

In [46]:
k1 = 0
k2 = 0
data = torch.rand((100,100,100)).type(dtype)
M = torch.rand((100,100)).type(dtype)
for graph1 in data:
    k2 = k1
    print(k1,k2)
    for graph2 in data[k2:]:
        M[k1,k2] = loss(graph1,graph2).item()
        k2 = k2 + 1
    k1 = k1 + 1

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [44]:
import numpy as np
import ot
data_np = np.random.random((100,100,100))
M_np = np.zeros((100,100))
k1 = 0
k2 = 0
for graph1 in data_np:
    k2 = k1
    for graph2 in data_np[k2:]:
        costs=ot.dist(graph1,graph2,metric='euclidean') 
        M_np[k1,k2] = ot.emd2([], [], costs)
        k2 = k2 + 1
    k1 = k1 + 1

In [42]:
M_np = M_np + M_np.T

In [43]:
M_np

array([[1.98682149e-08, 9.24455622e-01, 8.10288356e-01, ...,
        8.62579319e-01, 9.83293593e-01, 9.21770605e-01],
       [9.24455622e-01, 1.32454766e-08, 8.77677934e-01, ...,
        7.48265029e-01, 9.17399919e-01, 9.00358107e-01],
       [8.10288356e-01, 8.77677934e-01, 1.32454766e-08, ...,
        8.48757798e-01, 9.70896993e-01, 9.05877691e-01],
       ...,
       [8.62579319e-01, 7.48265029e-01, 8.48757798e-01, ...,
        6.62273831e-09, 8.91479243e-01, 9.16033050e-01],
       [9.83293593e-01, 9.17399919e-01, 9.70896993e-01, ...,
        8.91479243e-01, 2.72944261e-08, 9.74919142e-01],
       [9.21770605e-01, 9.00358107e-01, 9.05877691e-01, ...,
        9.16033050e-01, 9.74919142e-01, 2.92341813e-08]])